In [1]:
import pandas as pd
import psycopg2
from psycopg2 import sql

#EXTRACT
# Read the CSV data into a DataFrame
df = pd.read_csv("C:/Users/obafe/Downloads/Top 50 US Tech Companies 2022 - 2023.csv")

#TRANSFORM
def transform_data(dataframe):
    # Calculate a new column: Revenue per Employee
    dataframe['Revenue per Employee'] = dataframe['Annual Revenue 2022-2023 (USD in Billions)'] / dataframe['Employee Size']
    return dataframe
transformed_df = transform_data(df)

db_params = {
    'host': 'localhost',
    'dbname': 'Femi_Database',
    'user': 'postgres',
    'password': 'ODLawal12'
}

# Establish a connection to the PostgreSQL database
conn = psycopg2.connect(**db_params)

# Create a cursor
cur = conn.cursor()

# Create a new table for the data
table_name = 'companies'
create_table_query = """
    CREATE TABLE IF NOT EXISTS {} (
        Company_Name TEXT PRIMARY KEY,
        Sector TEXT,
        HQ_State TEXT,
        Founding_Year INTEGER,
        Annual_Revenue REAL,
        Market_Cap REAL,
        Stock_Name TEXT,
        Annual_Income_Tax REAL,
        Employee_Size INTEGER,
        Revenue_Per_Employee REAL
    );
""".format(table_name)
cur.execute(create_table_query)
conn.commit()

#LOAD
# Load data from transformed DataFrame to the PostgreSQL database
for index, row in transformed_df.iterrows():
    insert_query = sql.SQL("""
        INSERT INTO {} (company_name, sector, hq_state, founding_year, annual_revenue,
                        market_cap, stock_name, annual_income_tax, employee_size, revenue_per_employee)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """).format(sql.Identifier(table_name))
    
    data = (row['Company Name'], row['Sector'], row['HQ State'], row['Founding Year'],
            row['Annual Revenue 2022-2023 (USD in Billions)'],
            row['Market Cap (USD in Trillions)'], row['Stock Name'],
            row['Annual Income Tax in 2022-2023 (USD in Billions)'],
            row['Employee Size'], row['Revenue per Employee'])  # Use the calculated column
    
    cur.execute(insert_query, data)
    conn.commit()
